In [20]:
# do a hacky import since brainaudio requires python 3.12 and the .wfst env requires an older
# python version 
import importlib                                                                                                                                                                                                    
import wfst_utils
importlib.reload(wfst_utils)
from wfst_utils import build_lm_decoder, lm_decode, arrange_logits, _cer_and_wer, augment_nbest
import sys
import numpy as np
import re
import pickle
import pandas as pd
import sys                                                                                                                                                                                                          

In [21]:
dataset = 'b2t_24'
load_lm = True

In [40]:
if dataset == "b2t_25":
    blank_penalty = np.log(90)
    acoustic_scale = 0.325
    beam = 17
    rescore = True
    base_path = "/home/ebrahim/data2/brain2text/b2t_25/logits/baseline_rnn_ucd_seeds_1_to_9/"
    folder_name = [f"baseline_rnn_ucd_npl_seed_{i}" for i in range(1,10,1)]
    
if dataset == "b2t_24":
    blank_penalty = np.log(7)
    acoustic_scale = 0.5
    beam = 18
    rescore = True
    base_path = "/home/ebrahim/data2/brain2text/b2t_24/logits/bidirectional_gru_all/"
    #folder_name = [f"bidirectional_gru_seed_{i}" for i in range(0,10,1)]
    folder_name = ["bidirectional_gru_seed_0"]

lmDir = "/home/ebrahim/data2/brain2text/lm/speech_5gram/lang_test"
nbest_value = 100
return_n_best = True

In [38]:
import pandas as pd
nbest_zero = pd.read_pickle("/home/ebrahim/data2/brain2text/b2t_24/wfst_outputs/bidirectional_RNN/nbest_wfst_rescore_True_acoustic_scale_0.5_test_seed_0.pkl")
print(len(nbest_zero))

784


In [15]:

#folder_name = ["/home/ebrahim/data2/brain2text/b2t_25/logits/pretrained_RNN/"]

In [16]:
#logits_all = np.load("/home/ebrahim/data2/brain2text/b2t_25/logits/pretrained_RNN/logits_val.npz")

In [26]:
if load_lm and 'ngramDecoder' not in globals(): 
    
    ngramDecoder = build_lm_decoder(
        lmDir,
        acoustic_scale=acoustic_scale, #1.2
        nbest=nbest_value,
        beam=beam,
    )
    print("loaded LM")

In [31]:
beam

18

In [30]:
rescore_bool = True

for seed, fn in zip(seeds_list, folder_name):

    logits_all = np.load(f"{base_path}{fn}/logits_test.npz")

    print(len(logits_all.keys()))

    if dataset == "b2t_25":
        
        nbest_save_path = "/home/ebrahim/data2/brain2text/b2t_25/wfst_outputs/pretrained_RNN/nbest_wfst_rescore_{}_acoustic_scale_{}_test_seed_{}.pkl".format(rescore_bool, acoustic_scale, seed+1)
        nbest_augmented_save_path = "/home/ebrahim/data2/brain2text/b2t_25/wfst_outputs/pretrained_RNN/nbest_wfst_augmented_rescore_{}_acoustic_scale_{}_test_seed_{}.pkl".format(rescore_bool, acoustic_scale, seed+1)

    else:

        nbest_save_path = "/home/ebrahim/data2/brain2text/b2t_24/wfst_outputs/bidirectional_RNN/nbest_wfst_rescore_{}_acoustic_scale_{}_test_seed_{}.pkl".format(rescore_bool, acoustic_scale, seed)

        
    nbest_outputs = []
    nbest_augmented_outputs = []

    for trial_idx in range(len(logits_all.keys())):
        
        if trial_idx % 100 == 0:
            
            print(f"Decoding trial {trial_idx}...")
        
        logits = logits_all[f'arr_{trial_idx}']
            
        rearranged_logits = arrange_logits(logits)
        
        nbest = lm_decode(
                        ngramDecoder,
                        rearranged_logits[0],
                        blankPenalty=blank_penalty,
                        returnNBest=return_n_best,
                        rescore=rescore_bool,
                    )

        if dataset == 'b2t_25':
            
            nbest_augmented = augment_nbest(nbest, acoustic_scale=acoustic_scale, top_candidates_to_augment=20, score_penalty_percent=0.01)
            nbest_augmented_outputs.append(nbest_augmented)
            
            
        nbest_outputs.append(nbest)
        
        with open(nbest_save_path, 'wb') as f:
            pickle.dump(nbest_outputs, f)
            
        if dataset == 'b2t_25':
            with open(nbest_augmented_save_path, 'wb') as f:
                pickle.dump(nbest_augmented_outputs, f)

Decoding trial 0...
Decoding trial 100...
Decoding trial 200...
Decoding trial 300...
Decoding trial 400...
Decoding trial 500...
Decoding trial 600...
Decoding trial 700...
Decoding trial 800...
Decoding trial 900...
Decoding trial 1000...
Decoding trial 1100...


In [9]:
len(logits_all)

1200

In [ ]:
print("HELLO")

(237, 41)

In [24]:
rearranged_logits.shape

(1, 237, 41)